In [211]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MultiLabelBinarizer

In [212]:
# 향수 데이터셋 불러오기
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/capstone/data/data_preprocessing.csv', encoding='utf-8')
df.head(1)

,제품 번호,제품명,Top Note,Middle Note,Base Note,브랜드,부향률,가격,설명,Notes,Notes_list,multi_hot
0,0,겐조 플라워 바이 겐조 오 드 퍼퓸,"블랙커런트, 산사나무, 불가리안 로즈","오포포낙스, 자스민, 파라마 바이올렛, 로즈, 머스크","바닐라, 인센스",겐조,오 드 퍼퓸,"106,000",NaN,"블랙커런트, 산사나무, 불가리안 로즈,오포포낙스, 자스민, 파라마 바이올렛, 로즈,...","['로즈', '오포포낙스', '블랙커런트', '불가리안 로즈', '인센스', '파라...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [213]:
# 함수 note 유사도 계산을 위한 함수
def sim(a, b):
    score = 0
    a_list = df.at[a, 'multi_hot']
    b_list = df.at[b, 'multi_hot']
    for i in range(len(mlb.classes_)):
        if a_list[i]==1 and b_list[i]==1:
            score += 1
    return score

In [214]:
# itme list 만들기
item_list = []

for i in range(len(df)):
    item_list.append(i)

In [215]:
# empty 유사도 matrix 만들기
item_number = pd.DataFrame({
    'number' : item_list
})

columns = item_list
sim_matrix = pd.DataFrame(columns=columns, index=item_list)

sim_matrix = pd.concat([item_number, sim_matrix], axis=1)

sim_matrix.head(1)

,number,0,1,2,3,4,5,6,7,8,...,89,90,91,92,93,94,95,96,97,98
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [216]:
# item similarity matrix 점수 채우기
for i in range(len(item_list)):
    for j in range(len(item_list)):
        if i==j:
            sim_matrix.at[i,j] = -1
        else:
            sim_matrix.at[i,j] = sim(i,j)

In [217]:
sim_matrix

,number,0,1,2,3,4,5,6,7,8,...,89,90,91,92,93,94,95,96,97,98
0,0,-1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,-1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,-1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,94,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,-1,0,0,0,0
95,95,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-1,0,0,0
96,96,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-1,0,0
97,97,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1,0


In [218]:
# 유사한 향수 TopK matrix 만들기
item_number = pd.DataFrame({
    '제품 번호' : item_list
})

columns = ['TOP1', 'TOP2', 'TOP3', 'TOP4', 'TOP5', 'TOP6']
top_matrix = pd.DataFrame(columns=columns, index=item_list)

topk_matrix = pd.concat([item_number, top_matrix], axis=1)

topk_matrix.head(1)

,제품 번호,TOP1,TOP2,TOP3,TOP4,TOP5,TOP6
0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [219]:
# topk 추출 후 matrix에 저장
for i in range(len(item_list)):
    sim_matrix.sort_values(i, ascending=False, inplace=True)
    top = sim_matrix.reset_index(drop=True)
    
    # topk 추출
    top1 = top.at[0,'number']
    top2 = top.at[1,'number']
    top3 = top.at[2,'number']
    top4 = top.at[3,'number']
    top5 = top.at[4,'number']
    top6 = top.at[5,'number']
    
    # topk_matrix에 저장
    topk_matrix.at[i, 'TOP1'] = top1
    topk_matrix.at[i, 'TOP2'] = top2
    topk_matrix.at[i, 'TOP3'] = top3
    topk_matrix.at[i, 'TOP4'] = top4
    topk_matrix.at[i, 'TOP5'] = top5
    topk_matrix.at[i, 'TOP6'] = top6

In [220]:
topk_matrix.to_csv('/content/drive/MyDrive/Colab Notebooks/capstone/data/all_notes_topk.csv', index=False)